In [19]:
# importing the libraries
import pandas as pd
import numpy as np
import os
import skorch
from sklearn.model_selection import cross_val_predict
# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader

#Transformers
from module import *

In [2]:
# Load training data set (small amount to test if it works first)

xtrain = r"data/training_sample_NoSparse.csv.gz"
ytrain = r"data/training_label_NoSparse.csv.gz"
xtest = r"data/testing_sample_NoSparse.csv.gz"
ytest = r"data/testing_label_NoSparse.csv.gz"


In [3]:
#check balance
samplesdf = pd.DataFrame()
for df in  pd.read_csv(ytrain,compression ="gzip",delimiter=',', chunksize = 10000, header=0):
    samplesdf = samplesdf.append(df)
y_train = samplesdf.to_numpy()

num0 = 0
num1 = 0
for x in y_train:
    if x == 0:
        num0 = num0 + 1
    else:
        num1 = num1 + 1
num0,num1 #checking if it is balanced

(15161, 14094)

In [4]:
df = pd.read_csv(xtrain,compression ="gzip",delimiter=',', nrows=123, header=0)
x_train = df.to_numpy()
print(x_train)

[[0.   0.   0.   ... 3.11 0.   3.69]
 [0.   0.   0.   ... 3.16 0.   3.33]
 [0.   0.   0.   ... 2.98 0.   3.64]
 ...
 [1.39 0.   0.   ... 1.95 0.   3.57]
 [0.   2.08 0.   ... 2.71 0.   3.09]
 [0.   0.   0.   ... 3.12 0.   2.85]]


In [5]:
class Dataset(Dataset):

    def __init__(self,samples,labels,numrows):

        self.data = pd.read_csv(samples,compression ="gzip",delimiter=',', nrows = numrows, header=0)
        self.label = pd.read_csv(labels,compression ="gzip",delimiter=',', nrows = numrows, header=0)
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        rna = self.data.iloc[idx]
        seplb = self.label.iloc[idx]
        rna = np.array([rna])
        seplb = np.array([seplb])
        rna = rna.astype('double').reshape(-1,3273)
        i = 0
        d = np.zeros((len(seplb),2))
        for x in seplb:
            if x == 0:
                d[i] = [1,0]
            else:
                d[i] = [0,1]
        sample = {'rna': rna, 'label': d}

        return sample

In [6]:
train_dataset = Dataset(samples=xtrain,labels=ytrain,numrows = 29255)
test_dataset = Dataset(samples = xtest,labels = ytest,numrows = 3241)

In [7]:
# Hyper Parameters
EPOCH = 20             # train the training data n times, to save time, we just train 1 epoch
LR = 0.0001              # learning rate
batch_size = 32
wd = LR / EPOCH

In [8]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
class SmallSetTransformer(nn.Module):
    def __init__(self,):
        super().__init__()
        self.enc = nn.Sequential(
            SAB(dim_in=3273, dim_out=256, num_heads=4),
            SAB(dim_in=256, dim_out=256, num_heads=4),
        )
        self.dec = nn.Sequential(
            PMA(dim=256, num_heads=4, num_seeds=1),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=2),
        )

    def forward(self, x):
        x = self.enc(x)
        x = self.dec(x)
        return x.squeeze(-1)

In [10]:
trans = SmallSetTransformer()
optimizer = torch.optim.Adam(trans.parameters(), lr=LR)   
loss_func = nn.L1Loss()                      
if torch.cuda.is_available():
    loss_func = loss_func.cuda()
    trans = trans.cuda()
trans = trans.double()
print(trans)

SmallSetTransformer(
  (enc): Sequential(
    (0): SAB(
      (mab): MAB(
        (fc_q): Linear(in_features=3273, out_features=256, bias=True)
        (fc_k): Linear(in_features=3273, out_features=256, bias=True)
        (fc_v): Linear(in_features=3273, out_features=256, bias=True)
        (fc_o): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (1): SAB(
      (mab): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (fc_o): Linear(in_features=256, out_features=256, bias=True)
      )
    )
  )
  (dec): Sequential(
    (0): PMA(
      (mab): MAB(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (fc_o): Linear(in_features=256, out_fea

def test(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in train_loader:
            rna,labels = batch["rna"], batch["label"] 
            if torch.cuda.is_available():
                rna = rna.cuda()
                labels = labels.cuda()
            labels = labels.reshape(-1,2)
            outputs = model(rna)
            outputs = outputs.reshape(-1,2)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        TrainaccTest.append(correct/total)
           # print(total,correct)

        print('Train Accuracy of the model on the train rna: {} %'.format((correct / total) * 100))
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in test_loader:
            rna,labels = batch["rna"], batch["label"] 
            if torch.cuda.is_available():
                rna = rna.cuda()
                labels = labels.cuda()
            labels = labels.reshape(-1,2)
            outputs = model(rna)
            outputs = outputs.reshape(-1,2)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        TestaccTest.append(correct/total)
    #print(total,correct)

        print('Test Accuracy of the model on the test rna: {} %'.format((correct / total) * 100))



def train(model): 
    total_step = len(train_loader)
    num_epoch = EPOCH
    for epoch in range(num_epoch):
        model.train()
        for i, batch in enumerate(train_loader):
            # Run the forward pass
            rna,labels = batch["rna"], batch["label"] 
            if torch.cuda.is_available():
                rna = rna.cuda()
                labels = labels.cuda()
            outputs = model(rna)
            labels = labels.long()
            #print(outputs)
            outputs = outputs.reshape(-1,2)
            labels = labels.reshape(-1,2)
            loss = loss_func(outputs, labels)
            train_losses.append(loss.item())

            # Backprop and perform Adam optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)
            correct = (predicted == labels).sum().item()
            train_acc.append(correct / total)
            if i % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                        .format(epoch + 1, num_epoch , i + 1, total_step, loss.item(),
                                (correct / total) * 100))
        test(model)

In [13]:
net = NeuralNetClassifier(
    module=MyModule,
    train_split=None,
)



y_pred = cross_val_predict(net, X, y, cv=5)

Epoch [1/20], Step [1/915], Loss: 0.5954, Accuracy: 56.25%
Epoch [1/20], Step [101/915], Loss: 0.4059, Accuracy: 56.25%
Epoch [1/20], Step [201/915], Loss: 0.3782, Accuracy: 75.00%
Epoch [1/20], Step [301/915], Loss: 0.3930, Accuracy: 68.75%
Epoch [1/20], Step [401/915], Loss: 0.3767, Accuracy: 75.00%
Epoch [1/20], Step [501/915], Loss: 0.3853, Accuracy: 71.88%
Epoch [1/20], Step [601/915], Loss: 0.3670, Accuracy: 71.88%
Epoch [1/20], Step [701/915], Loss: 0.2932, Accuracy: 87.50%
Epoch [1/20], Step [801/915], Loss: 0.3498, Accuracy: 81.25%
Epoch [1/20], Step [901/915], Loss: 0.3608, Accuracy: 68.75%
Train Accuracy of the model on the train rna: 81.12459408648094 %
Test Accuracy of the model on the test rna: 77.19839555692688 %
Epoch [2/20], Step [1/915], Loss: 0.3578, Accuracy: 68.75%
Epoch [2/20], Step [101/915], Loss: 0.2917, Accuracy: 84.38%
Epoch [2/20], Step [201/915], Loss: 0.3092, Accuracy: 84.38%
Epoch [2/20], Step [301/915], Loss: 0.2728, Accuracy: 90.62%
Epoch [2/20], Step [

Epoch [12/20], Step [101/915], Loss: 0.1235, Accuracy: 90.62%
Epoch [12/20], Step [201/915], Loss: 0.0695, Accuracy: 96.88%
Epoch [12/20], Step [301/915], Loss: 0.0824, Accuracy: 96.88%
Epoch [12/20], Step [401/915], Loss: 0.0601, Accuracy: 96.88%
Epoch [12/20], Step [501/915], Loss: 0.0669, Accuracy: 96.88%
Epoch [12/20], Step [601/915], Loss: 0.0833, Accuracy: 96.88%
Epoch [12/20], Step [701/915], Loss: 0.0694, Accuracy: 96.88%
Epoch [12/20], Step [801/915], Loss: 0.0585, Accuracy: 96.88%
Epoch [12/20], Step [901/915], Loss: 0.0326, Accuracy: 100.00%
Train Accuracy of the model on the train rna: 95.45718680567424 %
Test Accuracy of the model on the test rna: 85.25146559703796 %
Epoch [13/20], Step [1/915], Loss: 0.0271, Accuracy: 100.00%
Epoch [13/20], Step [101/915], Loss: 0.0893, Accuracy: 93.75%
Epoch [13/20], Step [201/915], Loss: 0.0843, Accuracy: 96.88%
Epoch [13/20], Step [301/915], Loss: 0.0991, Accuracy: 90.62%
Epoch [13/20], Step [401/915], Loss: 0.0402, Accuracy: 100.00%
E